In [ ]:
# PRESET

%matplotlib inline

import matplotlib as mpl
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import pandas as pd
import scipy
from scipy import stats
import seaborn as sns
import os
from os import listdir
from matplotlib.lines import Line2D
from matplotlib import colors as mcolors
from matplotlib.ticker import PercentFormatter

import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(suppress=True)
np.set_printoptions(precision=4)
plt_style = 'seaborn-notebook'
pd.set_option('precision', 2)
pd.set_option('max_columns',10)

## To download 

# 1. IMPORT DATA

## For kapa 1.5x samples

In [ ]:
# direct to folder with dsDNA inserts files
%cd ./dsDNA_inserts

In [ ]:
# inputing all inserts files ... might take forever
kapa_insert_files = os.listdir()
kapa_insert_files.sort()
frames = []
newnames = []
for file in kapa_insert_files:
    if file.endswith("_dedup_inserts.txt"):
        frames.append(pd.read_csv(file, sep=',', names=[file]))
        name = file.split('_inserts')[0]
        newnames.append(name)
        
kapa_insert_df = pd.concat(frames, axis=1)

oldnames = kapa_insert_df.columns
finalnames = dict(zip(oldnames, newnames))
kapa_insert_df.rename(columns = finalnames, inplace=True)

In [ ]:
# check for all files
kapa_insert_df.columns

## For srsly 1.5x samples

In [ ]:
# direct to folder with ssDNA inserts files
%cd ..
%cd ./ssDNA_inserts

In [ ]:
# inputing all inserts files ... might take forever
srsly_insert_files = os.listdir()
srsly_insert_files.sort()
frames = []
newnames = []
for file in srsly_insert_files:
    if file.endswith("_dedup_inserts.txt"):
        frames.append(pd.read_csv(file, sep=',', names=[file]))
        name = file.split('_inserts')[0]
        newnames.append(name)
        
srsly_insert_df = pd.concat(frames, axis=1)

oldnames = srsly_insert_df.columns
finalnames = dict(zip(oldnames, newnames))
srsly_insert_df.rename(columns = finalnames, inplace=True)

In [ ]:
srsly_insert_df.columns

#    For size-selected samples

In [ ]:
# direct to folder with size-selected inserts files
%cd ..
%cd ./size_selected_inserts

In [ ]:
# inputing all inserts files ... might take forever
ranger_insert_files = os.listdir()
ranger_insert_files.sort()
frames = []
newnames = []
for file in ranger_insert_files:
    if file.endswith("_dedup_inserts.txt"):
        frames.append(pd.read_csv(file, sep=',', names=[file]))
        name = file.split('_inserts')[0]
        newnames.append(name)
        
ranger_insert_df = pd.concat(frames, axis=1)

oldnames = ranger_insert_df.columns
finalnames = dict(zip(oldnames, newnames))
ranger_insert_df.rename(columns = finalnames, inplace=True)

In [ ]:
# check for all files needed
ranger_insert_df.columns

# 2. Combining all human/virus data into one dataframe

In [ ]:
# function to pool all human inserts into one file
def human(species,dataframe):
    templst = []
    insert_df = dataframe.copy()
    file = insert_df.columns
    for i in file:
        temp = i.split('_')
        if temp[len(temp)-1] == 'dedup':
            if species == 'human':
                if temp[len(temp)-2] == 'human':
                    templst.append(i)
            elif species == 'virus':
                if temp[len(temp)-2] != 'human':
                    templst.append(i)
    df = insert_df[templst]
    return df

## dsDNA SET

HUMAN

In [ ]:
# get all human inserts from KAPA set into 1 col
human_kapa_df = human('human',kapa_insert_df)
print(human_kapa_df.columns)
human_kapa_df = human_kapa_df.T.stack().reset_index(name='Human')
human_kapa_df = human_kapa_df.drop(['level_0','level_1'], axis=1)

VIRUS

In [ ]:
# get all the virus inserts into 1 col of each species
all_adv_virus_kapa_df = kapa_insert_df[['ADV446-K-15x_S194_MK883610_dedup','ADV468-K-15x_S180_NC010956_dedup','ADV472-K-15x_S195_AC000018_dedup']]
adv_virus_kapa_df = all_adv_virus_kapa_df.T.stack().reset_index(name='ADV_kapa')
adv_virus_kapa_df = adv_virus_kapa_df.drop(['level_0','level_1'],axis=1)

all_hsv_virus_kapa_df = kapa_insert_df[['HSV089-K-15x_S17_L001_NC001798_masked_dedup','HSV369-K-15x_S18_L001_NC001798_masked_dedup']]
hsv_virus_kapa_df = all_hsv_virus_kapa_df.T.stack().reset_index(name='HSV_kapa')
hsv_virus_kapa_df = hsv_virus_kapa_df.drop(['level_0','level_1'],axis=1)

hhv_virus_kapa_df = kapa_insert_df[['HH6391-K-15x_S13_L001_AF157706_dedup']]
hhv_virus_kapa_df = hhv_virus_kapa_df.rename(columns={'HH6391-K-15x_S13_L001_AF157706_dedup':'HH6_kapa'})

all_ebv_virus_kapa_df = kapa_insert_df[['EBV057-K-15x_S177_NC007605_dedup','EBV152-K-15x_S179_NC007605_dedup','EBV158-K-15x_S192_NC007605_dedup']]
ebv_virus_kapa_df = all_ebv_virus_kapa_df.T.stack().reset_index(name='EBV_kapa')
ebv_virus_kapa_df = ebv_virus_kapa_df.drop(['level_0','level_1'],axis=1)

all_vzv_virus_kapa_df = kapa_insert_df[['VZV043-K-15x_S189_NC001348_masked_dedup','VZV062-K-15x_S190_NC001348_masked_dedup','VZV079-K-15x_S191_NC001348_masked_dedup']]
vzv_virus_kapa_df = all_vzv_virus_kapa_df.T.stack().reset_index(name='VZV_kapa')
vzv_virus_kapa_df = vzv_virus_kapa_df.drop(['level_0','level_1'],axis=1)

## ssDNA SET

HUMAN

In [ ]:
# get all human inserts from KAPA set into 1 col
human_srsly_df = human('human',srsly_insert_df)
print(human_srsly_df.columns)
human_srsly_df = human_srsly_df.T.stack().reset_index(name='Human')
human_srsly_df = human_srsly_df.drop(['level_0','level_1'], axis=1)

VIRUS

In [ ]:
# get all the virus inserts into 1 col of each species
srsly_virus_df = human('virus',srsly_insert_df)

#merge all ADV samples
all_adv_virus_srsly_df = srsly_virus_df.iloc[:,0:3]
print(all_adv_virus_srsly_df.columns)
adv_virus_srsly_df = all_adv_virus_srsly_df.T.stack().reset_index(name='ADV_srsly')
adv_virus_srsly_df = adv_virus_srsly_df.drop(['level_0','level_1'], axis=1)

#merge all EBV samples
all_ebv_virus_srsly_df = srsly_virus_df.iloc[:,3:6]
print(all_ebv_virus_srsly_df.columns)
ebv_virus_srsly_df = all_ebv_virus_srsly_df.T.stack().reset_index(name='EBV_srsly')
ebv_virus_srsly_df = ebv_virus_srsly_df.drop(['level_0','level_1'], axis=1)

#merge all VZV samples
all_vzv_virus_srsly_df = srsly_virus_df.iloc[:,6:9]
print(all_vzv_virus_srsly_df.columns)
vzv_virus_srsly_df = all_vzv_virus_srsly_df.T.stack().reset_index(name='VZV_srsly')
vzv_virus_srsly_df = vzv_virus_srsly_df.drop(['level_0','level_1'], axis=1)

## Size selection SET

HUMAN

In [ ]:
human_ranger_df = human('human',ranger_insert_df)
print(human_ranger_df.columns)
human_ranger_df = human_ranger_df.T.stack().reset_index(name='Human_ranger')
human_ranger_df = human_ranger_df.drop(['level_0','level_1'], axis=1)

VIRUS

In [ ]:
#merge all samples by virus type
adv_virus_ranger_df = ranger_insert_df[['ADV446-ranger_S1_L001_MK883610_dedup','ADV468-ranger_S2_L001_NC010956_dedup','ADV472-ranger_S3_L001_AC000018_dedup']]
adv_virus_ranger_df = adv_virus_ranger_df.T.stack().reset_index(name='ADV_ranger')
adv_virus_ranger_df = adv_virus_ranger_df.drop(['level_0','level_1'],axis=1)

ebv_virus_ranger_df = ranger_insert_df[['EBV057-ranger_S4_L001_NC007605_dedup','EBV152-ranger_S5_L001_NC007605_dedup','EBV158-ranger_S6_L001_NC007605_dedup']]
ebv_virus_ranger_df = ebv_virus_ranger_df.T.stack().reset_index(name='EBV_ranger')
ebv_virus_ranger_df = ebv_virus_ranger_df.drop(['level_0','level_1'],axis=1)

hsv_virus_ranger_df = ranger_insert_df[['HSV089-ranger_S10_L001_NC001798_masked_dedup','HSV369-ranger_S11_L001_NC001798_masked_dedup']]
hsv_virus_ranger_df = hsv_virus_ranger_df.T.stack().reset_index(name='HSV_ranger')
hsv_virus_ranger_df = hsv_virus_ranger_df.drop(['level_0','level_1'],axis=1)

hhv_virus_ranger_df = ranger_insert_df[['HH6391-ranger_S12_L001_AF157706_dedup']]
hhv_virus_ranger_df = hhv_virus_ranger_df.rename(columns={'HH6391-ranger_S12_L001_AF157706_dedup':'HH6_ranger'})

vzv_virus_ranger_df = ranger_insert_df[['VZV043-ranger_S7_L001_NC001348_masked_dedup','VZV062-ranger_S8_L001_NC001348_masked_dedup','VZV079-ranger_S9_L001_NC001348_masked_dedup']]
vzv_virus_ranger_df = vzv_virus_ranger_df.T.stack().reset_index(name='VZV_ranger')
vzv_virus_ranger_df = vzv_virus_ranger_df.drop(['level_0','level_1'],axis=1)

In [ ]:
cd ..

# 3. For Fig2 - dsDNA vs ssDNA

## Setting up dfs

In [ ]:
# for density plots, samples were merged by virus type
fig1adf = pd.concat([human_kapa_df, adv_virus_kapa_df, ebv_virus_kapa_df, vzv_virus_kapa_df], axis=1)
print(fig1adf.columns)   
fig2adf = pd.concat([human_srsly_df,adv_virus_srsly_df, ebv_virus_srsly_df, vzv_virus_srsly_df], axis=1)
print(fig2adf.columns)

In [ ]:
# for cummulaitve frequency plots, individual samples
fig1df = pd.concat([human_kapa_df,all_adv_virus_kapa_df, all_ebv_virus_kapa_df, all_vzv_virus_kapa_df], axis=1)
print(fig1df.columns)    
fig2df = pd.concat([human_srsly_df,all_adv_virus_srsly_df, all_ebv_virus_srsly_df, all_vzv_virus_srsly_df], axis=1)
print(fig2df.columns)    

In [ ]:
# getting location for ratio plots
fig10 =plt.figure(figsize=(6,3))

ax1 = fig10.add_subplot(121)
ax2 = fig10.add_subplot(122)
(n1, bins1, bars1) = ax1.hist(fig1adf, bins=range(501), density=True, cumulative=True, alpha=1, histtype='step',linewidth=1.5)
(n2, bins2, bars2) = ax2.hist(fig2adf, bins=range(501), density=True, cumulative=True, alpha=1, histtype='step',linewidth=1.5)

In [ ]:
# creating df for ratio plots
srs_ratio = pd.DataFrame({'Human':n2[0],'ADV':n2[3],'EBV':n2[2], 'VZV':n2[1]})
srs_ratio['VZV_ratio'] = srs_ratio.VZV/srs_ratio.Human
srs_ratio['EBV_ratio'] = srs_ratio.EBV/srs_ratio.Human
srs_ratio['ADV_ratio'] = srs_ratio.ADV/srs_ratio.Human
srs_ratio_plt = srs_ratio.iloc[:,4:]

kapa_ratio = pd.DataFrame({'Human':n1[0],'ADV':n1[3],'EBV':n1[2], 'VZV':n1[1]})
kapa_ratio['VZV_ratio'] = kapa_ratio.VZV/kapa_ratio.Human
kapa_ratio['EBV_ratio'] = kapa_ratio.EBV/kapa_ratio.Human
kapa_ratio['ADV_ratio'] = kapa_ratio.ADV/kapa_ratio.Human
kapa_ratio_plt = kapa_ratio.iloc[:,4:]

## dsDNA vs ssDNA plot

In [ ]:
# setting up figure basic
plt.rcParams["font.family"] = "arial"
plt.rcParams.update({'font.size': 8.5, 'xtick.color':'dimgrey', 'ytick.color':'dimgrey', 'figure.facecolor':'white'})

fig7 =plt.figure(figsize=(6.6,9))

ax1 = fig7.add_subplot(321)
ax2 = fig7.add_subplot(323)
ax3 = fig7.add_subplot(322)
ax4 = fig7.add_subplot(324)
ax5 = fig7.add_subplot(325)
ax6 = fig7.add_subplot(326)


########################
### for KAPA density ###
########################

# setting labels and colors
labels2 = [ 'VZV', 'EBV', 'ADV','Human']

colors2 = ['lightskyblue','limegreen','mediumorchid','darkorange']
colors2.reverse()

# histt2
ax1.hist(fig1adf, bins=range(501), density=True, cumulative=False, alpha=1, histtype='step',linewidth=1, color = colors2,label=labels2)
ax1.set_xlim(0,400)
ax1.set_ylim(0,0.027)
ax1.text(-0.1,1.15,'A',transform=ax1.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

# to change legend
handles2, l = ax1.get_legend_handles_labels()
new_handles2 = [Line2D([], [], c=h.get_edgecolor()) for h in handles2]
ax1.legend(handles = new_handles2, labels=labels2, ncol=1,frameon=False, loc='upper right')

# change ticks to percent
ax1.yaxis.set_major_formatter(PercentFormatter(1, decimals = 1,symbol=''))

# set xy label
ax1.set_xlabel('Fragment size',fontsize=10)
ax1.set_ylabel('Percent of reads',fontsize=10)

######################################
### for KAPA cummulative frequency ###
######################################

# setting labels and colors
labels = []
lst = fig1df.columns
for i in lst: 
    label = i.split('-')[0]
    labels.append(label)
print(labels)
labels.reverse()
print(labels)

colors = ['darkorange','slateblue','mediumorchid','orchid','limegreen','lightgreen','palegreen','lightskyblue','lightblue','paleturquoise']

# histtt
ax2.hist(fig1df, bins=range(1001), density=True, cumulative=True, alpha=1, histtype='step',linewidth=1.5, color = colors,label=labels)
ax2.set_xlim(0,400)
ax2.text(-0.1,1.15,'C',transform=ax2.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

# changing legend from box to line
handles, l = ax2.get_legend_handles_labels()
new_handles = [Line2D([], [], c=h.get_edgecolor()) for h in handles]
ax2.legend(handles=new_handles, labels=labels, ncol=1,frameon=False, loc='lower right')

# set xy label
ax2.set_xlabel('Fragment size',fontsize=10)
ax2.set_ylabel('Cumulative frequency',fontsize=10)

#########################
### for SRSLy density ###
#########################

# histt2
ax3.hist(fig2adf, bins=range(501), density=True, cumulative=False, alpha=1, histtype='step',linewidth=1, color = colors2,label=labels2)
ax3.set_xlim(0,400)
ax3.set_ylim(0,0.027)
ax3.text(-0.1,1.15,'B',transform=ax3.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

# to change legend
handles3, l3 = ax3.get_legend_handles_labels()
new_handles3 = [Line2D([], [], c=h.get_edgecolor()) for h in handles3]
ax3.legend(handles = new_handles3, labels=labels2, ncol=1,frameon=False, loc='upper right')

# change ticks to percent
ax3.yaxis.set_major_formatter(PercentFormatter(1, decimals = 1,symbol=''))

# set xy label
ax3.set_xlabel('Fragment size',fontsize=10)
ax3.set_ylabel('Percent of reads',fontsize=10)

#####################################
### for cummulative frequency fig ###
#####################################

# histtt
ax4.hist(fig2df, bins=range(1001), density=True, cumulative=True, alpha=1, histtype='step',linewidth=1.5, color = colors,label=labels)
ax4.set_xlim(0,400)
ax4.text(-0.1,1.15,'D',transform=ax4.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

# changing legend from box to line
handles4, l4 = ax4.get_legend_handles_labels()
new_handles4 = [Line2D([], [], c=h.get_edgecolor()) for h in handles4]
ax4.legend(handles=new_handles4, labels=labels, ncol=1,frameon=False, loc='lower right')

# set xy label
ax4.set_xlabel('Fragment size',fontsize=10)
ax4.set_ylabel('Cumulative frequency',fontsize=10)


#####################
### for ratio fig ###
#####################

colors5 = ['lightskyblue','limegreen','mediumorchid']

ax5 = kapa_ratio_plt.plot.line(ax=ax5, color=colors5)
ax5.set_xlabel('Fragment size',fontsize=10)
ax5.set_ylabel('Virus:Human Ratio',fontsize=10)
ax5.text(-0.1,1.15,'E',transform=ax5.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

ax6 = srs_ratio_plt.plot.line(ax=ax6, color=colors5)
ax6.set_xlabel('Fragment size',fontsize=10)
ax6.set_ylabel('Virus:Human Ratio',fontsize=10)
ax6.text(-0.1,1.15,'F',transform=ax6.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

# to change legend
ax5.legend(labels=['VZV','EBV','ADV'], ncol=1,frameon=False, loc='upper right')
ax5.set_xlim(0,400)
ax5.set_ylim(0,20)
ax6.legend(labels=['VZV','EBV','ADV'], ncol=1,frameon=False, loc='upper right')
ax6.set_xlim(0,400)
ax6.set_ylim(0,20)

sns.despine()
fig7.tight_layout()

plt.savefig('Fig2_dsDNA_vs_ssDNA.eps',dpi=300)

plt.show()


# 4. For Fig3 - Size selection

## Setting up dfs

In [ ]:
# Combine KAPA and ranger human data
human_df = pd.concat([human_kapa_df, human_ranger_df])

# Combine KAPA and ranger virus data
adv_df = pd.concat([adv_virus_kapa_df, adv_virus_ranger_df])
ebv_df = pd.concat([ebv_virus_kapa_df, ebv_virus_ranger_df])
hsv_df = pd.concat([hsv_virus_kapa_df, hsv_virus_ranger_df])
hhv_df = pd.concat([hhv_virus_kapa_df, hhv_virus_ranger_df])
vzv_df = pd.concat([vzv_virus_kapa_df, vzv_virus_ranger_df])

## Size selection plot

In [ ]:
## For size selecting plot
plt.rcParams["font.family"] = "arial"
plt.rcParams.update({'font.size':10, 'xtick.labelsize':8, 'ytick.labelsize':8,'xtick.color':'dimgrey', 'ytick.color':'dimgrey', 'figure.facecolor':'white'})

fig12 =plt.figure(figsize=(6.8,5))

colors = ['tan','indigo']

ax1 = fig12.add_subplot(231)
ax2 = fig12.add_subplot(232)
ax3 = fig12.add_subplot(233)
ax4 = fig12.add_subplot(234)
ax5 = fig12.add_subplot(235)
ax6 = fig12.add_subplot(236)

fig12.text(0.06, 0.5, 'Percentage of reads', ha='center', va='center',rotation='vertical')
fig12.text(0.5, 0.1, 'Fragment Size', ha='center', va='center')

ax1.hist(human_df, bins=range(501), density=True, cumulative=False, alpha=1, histtype='step',linewidth=1.5, color=colors)
ax1.set_ylim(0,0.03)
ax1.text(300,0.027,'Human', fontsize=10)
ax1.set_xticks(np.arange(0, 450,100))
ax1.yaxis.set_major_formatter(PercentFormatter(1, decimals = 1,symbol=''))

ax2.hist(adv_df, bins=range(501), density=True, cumulative=False, alpha=1, histtype='step',linewidth=1.5,color=colors, label=['Size Selection', 'Pre-Size Selection'])
ax2.set_ylim(0,0.03)
ax2.text(300,0.027,'ADV', fontsize=10)
ax2.set_xticks(np.arange(0, 450,100))
ax2.yaxis.set_major_formatter(PercentFormatter(1, decimals = 1,symbol=''))

ax3.hist(ebv_df, bins=range(501), density=True, cumulative=False, alpha=1, histtype='step',linewidth=1.5, color=colors)
ax3.set_ylim(0,0.03)
ax3.text(300,0.027,'EBV', fontsize=10)
ax3.set_xticks(np.arange(0, 450,100))
ax3.yaxis.set_major_formatter(PercentFormatter(1, decimals = 1,symbol=''))

ax4.hist(hhv_df, bins=range(501), density=True, cumulative=False, alpha=1, histtype='step',linewidth=1.5, color=colors)
ax4.set_ylim(0,0.03)
ax4.text(300,0.027,'HHV-6', fontsize=10)
ax4.set_xticks(np.arange(0, 450,100))
ax4.yaxis.set_major_formatter(PercentFormatter(1, decimals = 1,symbol=''))

ax5.hist(hsv_df, bins=range(501), density=True, cumulative=False, alpha=1, histtype='step',linewidth=1.5, color=colors)
ax5.set_ylim(0,0.03)
ax5.text(300,0.027,'HSV-2', fontsize=10)
ax5.set_xticks(np.arange(0, 450,100))
ax5.yaxis.set_major_formatter(PercentFormatter(1, decimals = 1,symbol=''))

ax6.hist(vzv_df, bins=range(501), density=True, cumulative=False, alpha=1, histtype='step',linewidth=1.5, color=colors)
ax6.set_ylim(0,0.03)
ax6.text(300,0.027,'VZV', fontsize=10)
ax6.set_xticks(np.arange(0, 450,100))
ax6.yaxis.set_major_formatter(PercentFormatter(1, decimals = 1,symbol=''))

fig12.subplots_adjust(bottom=0.2)

# changing legend from box to line
handles4, l4 = ax2.get_legend_handles_labels()
new_handles4 = [Line2D([], [], c=h.get_edgecolor()) for h in handles4]
ax5.legend(handles=new_handles4, labels=['Size Selection','Pre-Size Selection'], ncol=2,frameon=False, loc='lower center',prop={'size': 10}, bbox_to_anchor=(0.4,-0.6))
sns.despine();

plt.savefig('Fig3_size_selection.eps',dpi=300,facecolor=ax1.get_facecolor(), edgecolor='none')

# 5. For fold change fig

## import dfs

In [ ]:
folddf = pd.read_excel('fold_change.xlsx')
folddf = folddf.replace(regex={'HH6': 'HHV-6', 'HSV': 'HSV-2'})
folddf

## plotting

In [ ]:
palette = sns.color_palette("Set2",5)

plt.rcParams["font.family"] = "arial"
plt.rcParams.update({'font.size':12, 'xtick.labelsize':10, 'ytick.labelsize':10,'xtick.color':'dimgrey', 'ytick.color':'dimgrey', 'figure.facecolor':'white'})

fig13, (ax2, ax1) = plt.subplots(1, 2, figsize=(6.8, 3.4))

g = sns.scatterplot(x='percent_virus_preSS',y='percent_virus_postSS',data=folddf, hue='type',ax=ax1, palette=palette)
x = np.linspace(0, 15, 1000)
#ax1.set_ylim(0,100)
ax1.plot(x,x, color='gray', linestyle='--', linewidth=1)
ax1.set(xscale="log",yscale="log")
ax1.set_aspect('equal', adjustable='box')
ax1.legend(bbox_to_anchor=(1.3, 0), fontsize=11,loc='lower right', frameon=False, handletextpad=0,markerscale=0.75)
ax1.set_ylabel('Viral fraction post-selection', fontsize=11)
ax1.set_xlabel('Viral fraction pre-selection', fontsize=11)
ax1.text(-0.1,1.15,'B',transform=ax1.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

g = sns.stripplot(y='fold_enrichment',x='type',data=folddf,ax=ax2, palette=palette)
ax2.axhline(y=1, color='gray', linewidth=1.0, linestyle='--')
ax2.set_ylabel('Fold viral enrichment', fontsize=11)
ax2.set_xlabel('Virus', fontsize=12)
ax2.set_ylim(0,15)
ax2.text(-0.1,1.15,'A',transform=ax2.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

fig13.tight_layout()

sns.despine();

plt.savefig('Fig4_fold_change.eps',dpi=300)
plt.show()

# 6. For Chrx plot

In [ ]:
chrx_df = pd.read_excel('chrx_table.xlsx')
chrx_df_merge = chrx_df.groupby(['Chrx','sample','prep','dup']).sum().reset_index()
order = ['Chrx 1', 'Chrx 2', 'Chrx 3', 'Chrx 4', 'Chrx 5','Chrx 6','Chrx 7', 'Chrx 8','Chrx 9','Chrx 10','Chrx 11', 'Chrx 12','Chrx 13','Chrx 14', 'Chrx 15','Chrx 16','Chrx 17','Chrx 18','Chrx 19','Chrx 20','Chrx 21','Chrx 22','Chrx X','Chrx Y','Mito','Unlocalized']
mapping = {Chrx: i for i, Chrx in enumerate(order)}
key = chrx_df_merge['Chrx'].map(mapping) 
chrx_df_merge = chrx_df_merge.iloc[key.argsort()]
chrx_df_merge_dedup = chrx_df_merge[chrx_df_merge['dup']=='yes']
chrx_df_merge_no_dedup = chrx_df_merge[chrx_df_merge['dup']=='no']

In [ ]:
chrx_df_merge_dedup = chrx_df_merge_dedup.sort_values(by=['prep','Chrx'])
key = chrx_df_merge_dedup['Chrx'].map(mapping) 
chrx_df_merge_dedup = chrx_df_merge_dedup.iloc[key.argsort()]
chrx_df_merge_no_dedup = chrx_df_merge_no_dedup.sort_values(by=['prep','Chrx'])
key2 = chrx_df_merge_no_dedup['Chrx'].map(mapping) 
chrx_df_merge_no_dedup = chrx_df_merge_no_dedup.iloc[key2.argsort()]

In [ ]:
fig24,ax1 =plt.subplots(figsize=(6.8,3))

plt.rcParams["font.family"] = "arial"
plt.rcParams.update({'font.size': 9, 'xtick.labelsize':7, 'ytick.labelsize':7,'xtick.color':'black', 'ytick.color':'black', 'figure.facecolor':'white'})

bar1 = sns.barplot(ax=ax1,x='Chrx',y="percent",hue='prep', data=chrx_df_merge_no_dedup, palette=['#1A535C','#b30000'],ci=None)
bar2 = sns.barplot(ax=ax1,x='Chrx',y='percent',hue='prep', data=chrx_df_merge_dedup, palette=['#4ECDC4','#ef6548'],ci=None)

labels = ['Chrx 1', 'Chrx 2', 'Chrx 3', 'Chrx 4', 'Chrx 5','Chrx 6','Chrx 7', 'Chrx 8','Chrx 9','Chrx 10','Chrx 11', 'Chrx 12','Chrx 13','Chrx 14', 'Chrx 15','Chrx 16','Chrx 17','Chrx 18','Chrx 19','Chrx 20','Chrx 21','Chrx 22','Chrx X','Chrx Y','Unlocalized\nHuman','Mitochondria']

ax1.set_xlabel('')
ax1.set_xticklabels(labels = labels,rotation=70, color='k')
ax1.set_ylabel('Percentage of reads', fontsize=9)
ax1.set_ylim(0,9)

plt.legend(['Pre-size selection duplicate','Post-size selection duplicate','Pre-size selection human','Post-size selection human'],bbox_to_anchor=[0.9,1],frameon=False,ncol=1)

sns.despine()

fig24.subplots_adjust(bottom=0.25)

plt.savefig('Fig5_chrx.eps',dpi=300)

# 7. For Reads distribution plot

In [ ]:
distr_df = pd.read_excel('reads_distribution.xlsx')
ranger_df = distr_df[distr_df['prep']=='r']
kapa_df = distr_df[distr_df['prep']=='K']
srsly_df = distr_df[distr_df['prep']=='S']

In [ ]:
palette = sns.color_palette("Spectral",8)

plt.rcParams["font.family"] = "arial"
plt.rcParams.update({'font.size': 8.5,'xtick.labelsize':7, 'ytick.labelsize':7, 'xtick.color':'dimgrey', 'ytick.color':'dimgrey', 'figure.facecolor':'white'})

fig8,(ax1,ax2,ax3) =plt.subplots(1,3,figsize=(6.8,4), sharex=False, sharey=False)

bar1 = sns.barplot(ax=ax2,y='Sample',x='all',data=ranger_df, color=palette[4],label="Unclassifed reads")
bar2 = sns.barplot(ax=ax2, y="Sample",x="all-unmap",data=ranger_df,color=palette[2],label='Duplicate reads')
bar3 = sns.barplot(ax=ax2, y="Sample",x="all-unmap-dup",data=ranger_df,color=palette[0],label='Virus')
bar4 = sns.barplot(ax=ax2, y="Sample",x="all-unmap-dup-virus",data=ranger_df,color=palette[6],label='Mapped to Human')
ax2.text(-0.1,1.07,'B',transform=ax2.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

bar5 = sns.barplot(ax=ax1,y='Sample',x='all',data=kapa_df, color=palette[4],label="Unclassified")
bar6 = sns.barplot(ax=ax1, y="Sample",x="all-unmap",data=kapa_df, color=palette[2],label='Duplicate')
bar7 = sns.barplot(ax=ax1, y="Sample",x="all-unmap-dup",data=kapa_df, color=palette[0],label='Virus')
bar8 = sns.barplot(ax=ax1, y="Sample",x="all-unmap-dup-virus",data=kapa_df, color=palette[6],label='Human')
ax1.text(-0.1,1.07,'A',transform=ax1.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

bar9 = sns.barplot(ax=ax3,y='Sample',x='all',data=srsly_df, color=palette[4],label="Unclassified")
bar10 = sns.barplot(ax=ax3, y="Sample",x="all-unmap",data=srsly_df, color=palette[2],label='Duplicate')
bar11 = sns.barplot(ax=ax3, y="Sample",x="all-unmap-dup",data=srsly_df, color=palette[0],label='Virus')
bar12 = sns.barplot(ax=ax3, y="Sample",x="all-unmap-dup-virus",data=srsly_df, color=palette[6],label='Human')
ax3.text(-0.1,1.07,'C',transform=ax3.transAxes,fontsize=14, fontweight='bold', va='top', ha='right')

ax2.set_ylabel('')
ax3.set_ylabel('')
ax1.set_ylabel('Sample', rotation=90, fontsize=10)
ax2.set_xlabel('Percentage of reads', fontsize=10)
ax3.set_xlabel('')
ax1.set_xlabel('')
#ax1.set_xlabel('Percentage of reads',fontsize=10)

handlesz, labelsz = ax1.get_legend_handles_labels()
handlesz.reverse()
labelsz.reverse()

fig8.tight_layout()

lgd = plt.legend(handlesz, labelsz,bbox_to_anchor=[-0.8,-0.25],ncol=4, loc='lower center',frameon=False,handletextpad=0.3,columnspacing=2.2, fontsize=10)

fig8.subplots_adjust(bottom=0.2)

sns.despine()

plt.savefig('FigS1_reads_distribution.eps',dpi=300,facecolor=ax1.get_facecolor(), edgecolor='none')

# 8. For Mitochondria distribution plot

## setting dfs 

In [ ]:
# direct to folder with ssDNA inserts files again
%cd ./ssDNA_inserts

In [ ]:
# inputing all mito inserts files ... might take forever
mito_insert_files = os.listdir()
mito_insert_files.sort()
frames = []
newnames = []
for file in mito_insert_files:
    if file.endswith("dedup_mito_inserts.txt"):
        frames.append(pd.read_csv(file, sep=',', names=[file]))
        name = file.split('mito_inserts')[0]
        newnames.append(name)
mito_insert_df = pd.concat(frames, axis=1)
oldnames = mito_insert_df.columns
finalnames = dict(zip(oldnames, newnames))
mito_insert_df.rename(columns = finalnames, inplace=True)

mito_ssdna_df = mito_insert_df.T.stack().reset_index(name='ssDNA_mito')
mito_ssdna_df = mito_ssdna_df.drop(['level_0','level_1'], axis=1)

In [ ]:
# direct to folder with dsDNA inserts files again
%cd ..
%cd ./dsDNA_inserts

In [ ]:
# inputing all mito inserts files ... might take forever
mito_insert_files = os.listdir()
mito_insert_files.sort()
frames = []
newnames = []
for file in mito_insert_files:
    if file.endswith("dedup_mito_inserts.txt"):
        frames.append(pd.read_csv(file, sep=',', names=[file]))
        name = file.split('mito_inserts')[0]
        newnames.append(name)
mito_insert_df = pd.concat(frames, axis=1)
oldnames = mito_insert_df.columns
finalnames = dict(zip(oldnames, newnames))
mito_insert_df.rename(columns = finalnames, inplace=True)

mito_dsdna_df = mito_insert_df.T.stack().reset_index(name='dsDNA_mito')
mito_dsdna_df = mito_dsdna_df.drop(['level_0','level_1'], axis=1)

In [ ]:
mito_dedup_df = pd.concat([mito_dsdna_df, mito_ssdna_df],axis=1)

## plotting

In [ ]:
plt.rcParams["font.family"] = "arial"
plt.rcParams.update({'font.size':12, 'xtick.labelsize':10, 'ytick.labelsize':10,'xtick.color':'dimgrey', 'ytick.color':'dimgrey', 'figure.facecolor':'white'})

fig47 =plt.figure(figsize=(5,3))

ax1 = fig47.add_subplot(111)
ax2 = fig47.add_subplot(122)
(nw,barsw,binsw) = ax1.hist(kapa_insert_df['count'], bins=range(501), density=True, cumulative=False, alpha=1, histtype='step',linewidth=1.5)

ax1.set_ylim(0,0.02)
ax1.set_xticks(np.arange(0, 450,100))
ax1.yaxis.set_major_formatter(PercentFormatter(1, decimals = 1,symbol=''))

# changing legend from box to line
handles7, l7 = ax1.get_legend_handles_labels()
new_handles7 = [Line2D([], [], c=h.get_edgecolor()) for h in handles7]
ax1.legend(handles=new_handles7, labels=['ssDNA library preparation','dsDNA library preparation'], ncol=1,frameon=False, loc='upper right',prop={'size': 10})

sns.despine();
plt.savefig('FigS2_mito_fragments.eps')
plt.show()
